In [ ]:
create schema `nodejs` default character set utf8mb4 default collate utf8mb4_general_ci;

In [1]:
use nodejs;

SyntaxError: invalid syntax (1317822847.py, line 1)

In [ ]:
create table nodejs.users (
	id int not null auto_increment,
    name varchar(20) not null,
    age int	unsigned not null,
    married tinyint not null,
    comment text null,
    create_at datetime not null default now(),
    primary	key(id),
    unique index name_unique (name asc)
)

In [ ]:
DESC user;

In [ ]:
create table nodejs.comments (
id int not null auto_increment,
commenter int not null,
comment varchar(100) not null,
created_at datetime not null default now(),

primary key(id),
index commenter_idx(commenter asc),
constraint commenter foreign key (commenter) references nodejs.users (id)
on delete cascade
on update cascade
)
comment="댓글"
Engine=InnoDB;

In [ ]:
select * from users
select * from comments

In [ ]:
insert into users (name,age,married,comment) values ("test",20,0,"자기소개1")

In [ ]:
select * from users where age<30;

In [ ]:
select id,name from users order by age desc

In [ ]:
select id,name from users order by age desc limit 1 offset 1;

In [ ]:
update users set comment = "바꿀내용" where id=2;

In [ ]:
update users set comment = "변경내용" where id=1;

In [ ]:
delete from users where id=2;

### Sequelize ORM
- SQL 작업을 쉽게 할 수 있도록 도와주는 라이브러리
- ORM : Object Relational Mapping
- MySQL 외데오 다른 RDB에도 호환

In [3]:
%mkdir sequelize

In [4]:
%cd "sequelize"

c:\project_YJ\dev\projects\front\day19\sequelize


In [5]:
!npm i express morgan nunjucks sequelize sequelize-cli mysql2


added 202 packages in 7s

30 packages are looking for funding
  run `npm fund` for details


In [6]:
!npm i -D nodemon


added 27 packages, and audited 230 packages in 1s

34 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities


In [9]:
%ls

 C ����̺��� �������� �̸��� �����ϴ�.
 ���� �Ϸ� ��ȣ: 00F2-871C

 c:\project_YJ\dev\projects\front\day19\sequelize ���͸�

2024-07-30  14:17    <DIR>          .
2024-07-30  14:17    <DIR>          ..
2024-07-30  14:18    <DIR>          node_modules
2024-07-30  14:18               240 package.json
2024-07-30  14:18            90,616 package-lock.json
               2�� ����              90,856 ����Ʈ
               3�� ���͸�  106,416,406,528 ����Ʈ ����


In [10]:
!npx sequelize init


Sequelize CLI [Node: 20.15.0, CLI: 6.6.2, ORM: 6.37.3]

Created "config\config.json"
Successfully created models folder at "c:\project_YJ\dev\projects\front\day19\sequelize\models".
Successfully created migrations folder at "c:\project_YJ\dev\projects\front\day19\sequelize\migrations".
Successfully created seeders folder at "c:\project_YJ\dev\projects\front\day19\sequelize\seeders".


In [11]:
# %load "config/config.json"
{
  "development": {
    "username": "root",
    "password": null,
    "database": "database_development",
    "host": "127.0.0.1",
    "dialect": "mysql"
  },
  "test": {
    "username": "root",
    "password": null,
    "database": "database_test",
    "host": "127.0.0.1",
    "dialect": "mysql"
  },
  "production": {
    "username": "root",
    "password": null,
    "database": "database_production",
    "host": "127.0.0.1",
    "dialect": "mysql"
  }
}


In [12]:
%%writefile "config/config.json"
{
  "development": {
    "username": "root",
    "password": null, # 여기랑...근데 비밀번호니까 일단은...
    "database": "nodejs", # 여기 변경
    "host": "127.0.0.1",
    "dialect": "mysql"
  },
  "test": {
    "username": "root",
    "password": null,
    "database": "database_test",
    "host": "127.0.0.1",
    "dialect": "mysql"
  },
  "production": {
    "username": "root",
    "password": null,
    "database": "database_production",
    "host": "127.0.0.1",
    "dialect": "mysql"
  }
}


Overwriting config/config.json


In [13]:
%%writefile app.js
const express = require('express');
const path = require('path');
const morgan = require('morgan');
const nunjucks = require('nunjucks');

const { sequelize } = require('./models');

const app = express();
app.set('port', process.env.PORT || 3001);
app.set('view engine', 'html');
nunjucks.configure('views', {
  express: app,
  watch: true,
});
sequelize.sync({ force: false })
  .then(() => {
    console.log('데이터베이스 연결 성공');
  })
  .catch((err) => {
    console.error(err);
  });

app.use(morgan('dev'));
app.use(express.static(path.join(__dirname, 'public')));
app.use(express.json());
app.use(express.urlencoded({ extended: false }));

app.use((req, res, next) => {
  const error =  new Error(`${req.method} ${req.url} 라우터가 없습니다.`);
  error.status = 404;
  next(error);
});

app.use((err, req, res, next) => {
  res.locals.message = err.message;
  res.locals.error = process.env.NODE_ENV !== 'production' ? err : {};
  res.status(err.status || 500);
  res.render('error');
});

app.listen(app.get('port'), () => {
  console.log(app.get('port'), '번 포트에서 대기 중');
});

Writing app.js


In [14]:
# %load "package.json"
{
  "dependencies": {
    "express": "^4.19.2",
    "morgan": "^1.10.0",
    "mysql2": "^3.11.0",
    "nunjucks": "^3.2.4",
    "sequelize": "^6.37.3",
    "sequelize-cli": "^6.6.2"
  },
  "devDependencies": {
    "nodemon": "^3.1.4"
  }
}


In [15]:
%%writefile "package.json"
{
  "dependencies": {
    "express": "^4.19.2",
    "morgan": "^1.10.0",
    "mysql2": "^3.11.0",
    "nunjucks": "^3.2.4",
    "sequelize": "^6.37.3",
    "sequelize-cli": "^6.6.2"
  },
  "devDependencies": {
    "nodemon": "^3.1.4"
  },
  "main": "app.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1",
    "start": "nodemon app"
  }
}


Overwriting package.json


In [16]:
// %%writefile models/user.js
const Sequelize = require('sequelize')

class User extends Sequelize.Model {
  static initiate(sequelize) {
    User.init({
      name: {
        type: Sequelize.STRING(20),
        allowNull: false,
        unique: true
      },
      age: {
        type: Sequelize.INTEGER.UNSIGNED,
        allowNull: false,
      },
      married: {
        type: Sequelize.BOOLEAN,
        allowNull: false,
      },
      comment: {
        type: Sequelize.TEXT,
        allowNull: true,
      },
      created_at: {
        type: Sequelize.DATE,
        allowNull: false,
        defaultValue: Sequelize.NOW,
      },
    }, {
      sequelize,
      timestamps: false,
      underscored: false,
      modelName: 'User',
      tableName: 'users',
      paranoid: false, // true 면 deletedAt 컬럼 생생, 로우 복구를 위해 완전히 삭제않고 deletedAt에 표현
      charset: 'utf8',
      collate: 'utf8_general_ci',
    });
  }
  static associate(db) {}
};

module.exports = User;


Writing models/user.js


1. **initiate 메서드**
   - **명령어**: `static initiate(sequelize)`
   - **의미**: `db.sequelize` 객체를 전달하여 모델을 연결합니다.

2. **timestamps**
   - **명령어**: `timestamps: false`
   - **의미**: 자동으로 `createdAt`과 `updatedAt` 컬럼을 추가하지 않습니다.

3. **underscored**
   - **명령어**: `underscored: true`
   - **의미**: 컬럼명을 캐멀 케이스 대신 스네이크 케이스로 사용합니다 (예: `created_at`).

4. **modelName**
   - **명령어**: `modelName: 'User'`
   - **의미**: 모델의 이름을 설정하여 노드 프로젝트에서 사용합니다.

5. **tableName**
   - **명령어**: `tableName: 'users'`
   - **의미**: 데이터베이스 테이블 이름을 설정합니다. 기본적으로 모델 이름의 소문자 및 복수형 형태입니다.

6. **paranoid**
   - **명령어**: `paranoid: true`
   - **의미**: `deletedAt` 컬럼을 추가하여 로우 삭제 시 완전히 지우지 않고 복원할 수 있게 합니다.

7. **charset과 collate**
   - **명령어**: `charset: 'utf8', collate: 'utf8_general_ci'`
   - **의미**: 한글 입력을 지원합니다. 
   - **명령어**: `charset: 'utf8mb4', collate: 'utf8mb4_general_ci'`
   - **의미**: 이모티콘 입력을 지원합니다.

In [19]:
%%writefile models/comment.js
const Sequelize = require('sequelize');

class Comment extends Sequelize.Model {
  static initiate(sequelize) {
    Comment.init({
      comment: {
        type: Sequelize.STRING(100),
        allowNull: false,
      },
      created_at: {
        type: Sequelize.DATE,
        allowNull: true,
        defaultValue: Sequelize.NOW,
      },
    }, {
      sequelize,
      timestamps: false,
      modelName: 'Comment',
      tableName: 'comments',
      paranoid: false,
      charset: 'utf8mb4',
      collate: 'utf8mb4_general_ci',
    });
  }

  static associate(db) {
    db.Comment.belongsTo(db.User, { foreignKey: 'commenter', targetKey: 'id' });
  }
};

module.exports = Comment;

Writing models/comment.js


In [20]:
// %load models/index.js
'use strict';

const fs = require('fs');
const path = require('path');
const Sequelize = require('sequelize');
const process = require('process');
const basename = path.basename(__filename);
const env = process.env.NODE_ENV || 'development';
const config = require(__dirname + '/../config/config.json')[env];
const db = {};

let sequelize;
if (config.use_env_variable) {
  sequelize = new Sequelize(process.env[config.use_env_variable], config);
} else {
  sequelize = new Sequelize(config.database, config.username, config.password, config);
}

fs
  .readdirSync(__dirname)
  .filter(file => {
    return (
      file.indexOf('.') !== 0 &&
      file !== basename &&
      file.slice(-3) === '.js' &&
      file.indexOf('.test.js') === -1
    );
  })
  .forEach(file => {
    const model = require(path.join(__dirname, file))(sequelize, Sequelize.DataTypes);
    db[model.name] = model;
  });

Object.keys(db).forEach(modelName => {
  if (db[modelName].associate) {
    db[modelName].associate(db);
  }
});

db.sequelize = sequelize;
db.Sequelize = Sequelize;

module.exports = db;


In [22]:
%%writefile models/index.js
'use strict';

const fs = require('fs');
const path = require('path');
// const Sequelize = require('sequelize');

/////insert code
const Sequelize = require('sequelize');
const User = require('./user');
const Comment = require('./comment');
////

const process = require('process');
const basename = path.basename(__filename);
const env = process.env.NODE_ENV || 'development';
const config = require(__dirname + '/../config/config.json')[env];
const db = {};

let sequelize;
if (config.use_env_variable) {
  sequelize = new Sequelize(process.env[config.use_env_variable], config);
} else {
  sequelize = new Sequelize(config.database, config.username, config.password, config);
}

fs
  .readdirSync(__dirname)
  .filter(file => {
    return (
      file.indexOf('.') !== 0 &&
      file !== basename &&
      file.slice(-3) === '.js' &&
      file.indexOf('.test.js') === -1
    );
  })
  .forEach(file => {
    // const model = require(path.join(__dirname, file))(sequelize, Sequelize.DataTypes);
    // db[model.name] = model;
    
    ///에러 발생해서 변경
    const model = require(path.join(__dirname, file))
    if (model && typeof model.initiate === "function") {
      model.initiate(sequelize)
      db[model.name] = model;  
    }
    
    ///
  });

Object.keys(db).forEach(modelName => {
  if (db[modelName].associate) {
    db[modelName].associate(db);
  }
});

db.sequelize = sequelize;
db.Sequelize = Sequelize;

/////insert code
db.User = User;
db.Comment = Comment;

User.initiate(sequelize);
Comment.initiate(sequelize);

User.associate(db);
Comment.associate(db);
/////

module.exports = db;

Overwriting models/index.js


-  테이블 네임 변경 후(없는 테이블의 경우)

In [ ]:
Executing (default): SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_NAME = 'member' AND TABLE_SCHEMA = 'nodejs'

Executing (default): SHOW INDEX FROM `member` FROM `nodejs`

Executing (default): SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_NAME = 'comments2' AND TABLE_SCHEMA = 'nodejs'        

Executing (default): CREATE TABLE IF NOT EXISTS `comments2` (`id` INTEGER NOT NULL 
auto_increment , `comment` VARCHAR(100) NOT NULL, `created_at` DATETIME, `commenter` INTEGER, PRIMARY KEY (`id`), FOREIGN KEY (`commenter`) REFERENCES `member` (`id`) ON DELETE SET NULL ON UPDATE CASCADE) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_general_ci;

Executing (default): SHOW INDEX FROM `comments2` FROM `nodejs`

-  테이블 네임 변경 전(있는 테이블의 경우)

In [ ]:
Executing (default): SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_NAME = 'users' AND TABLE_SCHEMA = 'nodejs'

Executing (default): SHOW INDEX FROM `users` FROM `nodejs`

Executing (default): SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_NAME = 'comments' AND TABLE_SCHEMA = 'nodejs'

Executing (default): SHOW INDEX FROM `comments` FROM `nodejs`